# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.2. [**Ajustement des hyper-paramètres d'un estimateur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tuning-the-hyper-parameters-of-an-estimator)<br/>([_Tuning the hyper-parameters of an estimator_](https://scikit-learn.org/stable/model_selection.html#tuning-the-hyper-parameters-of-an-estimator))

# Sommaire

- **Volume** : 18 pages, 9 exemples, 3 papiers
- 3.2.1. [**Recherche exhaustive en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#exhaustive-grid-search)<br/>([_Exhaustive Grid Search_](https://scikit-learn.org/stable/model_selection.html#exhaustive-grid-search))
- 3.2.2. [**Optimisation aléatoire des paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#randomized-parameter-optimization)<br/>([_Randomized Parameter Optimization_](https://scikit-learn.org/stable/model_selection.html#randomized-parameter-optimization))
- 3.2.3. [**Recherche de paramètres optimaux par halving successifs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#searching-for-optimal-parameters-with-successive-halving)<br/>([_Searching for optimal parameters with successive halving_](https://scikit-learn.org/stable/model_selection.html#searching-for-optimal-parameters-with-successive-halving))
- 3.2.4. [**Conseils pour la recherche de paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tips-for-parameter-search)<br/>([_Tips for parameter search_](https://scikit-learn.org/stable/model_selection.html#tips-for-parameter-search))
- 3.2.5. [**Alternatives à la recherche de paramètres par force brute**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#alternatives-to-brute-force-parameter-search)<br/>([_Alternatives to brute force parameter search_](https://scikit-learn.org/stable/model_selection.html#alternatives-to-brute-force-parameter-search))